In [1]:
from pdf2image import convert_from_path

In [23]:
pages=convert_from_path(r'C:\\Medical Project\\backend\\resources\\patient_details\\pd_2.pdf',poppler_path=r'C:/poppler-23.01.0/Library/bin')

In [24]:
pages[0].show()
pages[1].show()

In [25]:
import pytesseract
import numpy as np
import cv2
from PIL import Image

In [26]:
def preprocess_image(img):
    gray=cv2.cvtColor(np.array(img),cv2.COLOR_BGR2GRAY)
    resized=cv2.resize(gray,None,fx=1.5,fy=1.5, interpolation=cv2.INTER_LINEAR)
    processed_image=cv2.adaptiveThreshold(
    resized,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,
    61,11)
    
    return processed_image

In [27]:
img=preprocess_image(pages[0])
Image.fromarray(img).show()

In [28]:
pytesseract.pytesseract.tesseract_cmd=r'C:\Program Files\Tesseract-OCR\tesseract.exe'
text=pytesseract.image_to_string(img,lang='eng')
print(text)

Patient Medical Record

Patient Information Birth Date

Jerry Lucas May 2 1998

(279) 920-8204 Weight:

4218 Wheeler Ridge Dr 57

Buffalo, New York, 14201 Height:

United States gnt
170

In Case of Emergency

- eee

Joe Lucas . 4218 Wheeler Ridge Dr
Buffalo, New York, 14201
Home phone United States
Work phone

General Medical History

Chicken Pox (Varicelia): Measles: ..

IMMUNE NOT IMMUNE

Have you had the Hepatitis B vaccination?

‘Yes

| List any Medical Problems (asthma, seizures, headaches):
N/A

7?
v

17/12/2020


In [29]:
import re

pattern = 'Patient Information(.*?)\(\d{3}\)'

matches = re.findall(pattern, text, flags=re.DOTALL)
matches

[' Birth Date\n\nJerry Lucas May 2 1998\n\n']

In [30]:
matches[0].strip()

'Birth Date\n\nJerry Lucas May 2 1998'

In [31]:
match = matches[0].replace("Birth Date","").strip()
match

'Jerry Lucas May 2 1998'

In [32]:
pattern = '((Jan|Feb|March|April|May|June|July|Aug|Sep|Oct|Nov|Dec)[ \d]+)'

date_matches = re.findall(pattern, match)
date = date_matches[0][0]
date

'May 2 1998'

In [33]:
match.replace(date, '').strip()

'Jerry Lucas'

In [34]:
def remove_noise_from_name(name):
    name = name.replace('Birth Date','').strip()
    date_pattern = '((Jan|Feb|March|April|May|June|July|Aug|Sep|Oct|Nov|Dec)[ \d]+)'
    date_matches = re.findall(date_pattern, name)
    
    if date_matches:
        date = date_matches[0][0]
        name = name.replace(date, '').strip()
    
    return name

In [36]:
name = '\n\n \n\n \n\nBirth Date\nJerry Lucas May 6 1972\n'

name = remove_noise_from_name(name)
name

'Jerry Lucas'

In [38]:
pattern = 'Patient Information(.*?)(\(\d{3}\) \d{3}-\d{4})'

matches = re.findall(pattern, text, flags=re.DOTALL)
matches[0][1]

'(279) 920-8204'

In [39]:
pattern = 'Have you had the Hepatitis B vaccination\?.*(Yes|No)'

matches = re.findall(pattern, text, flags=re.DOTALL)
matches

['Yes']

In [40]:
pattern = 'List any Medical Problems \(asthma, seizures, headaches\):(.*)'

matches = re.findall(pattern, text, flags=re.DOTALL)
matches[0].strip()

'N/A\n\n7?\nv\n\n17/12/2020'